# Useful functions

In [ ]:
#!/usr/bin/env python

# Submit

import os
import sys

def submit(coms, split_size):    
    os.system("split -l %s %s %s" % (split_size, coms, coms + "_split_"))
    os.system("chmod u+x %s" % coms + "_split_*")
    mcoms = [x for x in os.listdir("./") if coms + "_split" in x]
    with open("submission_" + coms, "w") as f:
        for mcom in mcoms:
            f.write("nohup ./%s &\n" % mcom)        
    os.system("chmod u+x %s" % "submission_" + coms)

args = sys.argv
if len(args) == 1:
    print("Usage: submit com_file split_size")
else:
    _, coms, split_size = args
    submit(coms, split_size)

In [1]:
#!/usr/bin/env python

# Filter alignment

import os
import sys

def do_filter():
    for h, seq in fasta_parser(aln):
        lenal = len(seq)
        if lenal <= float(length):
            break
        gaps = float(seq.count("-"))
        if gaps / lenal >= float(gappiness):
            continue
        else:
            print(">" + h)
            print(seq)
def fasta_parser(myfile):
    with open(myfile) as f:

        header = ""
        seq = ""
        for line in f:
            if line[0] == ">":
                if seq != "":
                    yield (header[1:], seq)
                    header = line.strip()
                    seq = ""
                else:
                    header = line.strip()
            else:
                seq += line.strip()
        yield (header[1:], seq)


args = sys.argv
if len(args) != 4:
    print("Usage: filter_alignment maximumgappiness minimumlength")
    print("E.g:  filter FAM123.aln 0.8 100")
    print("Filters out sequences with more than 80% of gaps, and shorther than 100 aas")
else:
    _, aln, gappiness, length = args
    do_filter()



Usage: filter alignment maximumgappiness minimumlength
E.g:  filter FAM123.aln 0.8 100
Filters out sequences with more than 80% of gaps, and shorther than 100 aas


In [ ]:
#!/usr/bin/env python

# check_quality

import os
import sys

def check_quality():
    allgaps = list()
    nseqs = 0
    nsps = set()
    for h, seq in fasta_parser(aln):
        
        lenal = len(seq)
        if lenal == 0:
            gappiness = 1
        else:
            nsps.add(h.split("_")[0])
            gaps = float(seq.count("-"))        
            gappiness = gaps / lenal
            nseqs +=1
        allgaps.append(gappiness)
        
    mline = "\t".join(map(str,[aln, lenal, nseqs, len(nsps), sum(allgaps) / len(allgaps), min(allgaps), max(allgaps)]))
    print(mline)
        
def fasta_parser(myfile):
    with open(myfile) as f:

        header = ""
        seq = ""
        for line in f:
            if line[0] == ">":
                if seq != "":
                    yield (header[1:], seq)
                    header = line.strip()
                    seq = ""
                else:
                    header = line.strip()
            else:
                seq += line.strip()
        yield (header[1:], seq)


args = sys.argv
if len(args) != 2:
    print("Usage: check_quality alignment ")
    print("Output: name of alignment len_alignment number_of_sequences mean_gappiness min_gappiness max_gappiness")

else:
    _, aln = args
    check_quality()



In [ ]:
looper aln 0 before after 

In [ ]:
#!/usr/bin/env python
import os
import sys

        
def looper(kw, nkw, bf, af):
    
    if nkw == "0":
        mfiles = [x for x in os.listdir("./") if kw in x]
    else:
        mfiles = [x for x in os.listdir("./") if kw in x and nkw not in x]
    
    if bf == "0" and af != "0":
        for f in mfiles:
            print(" ".join([f, af]))
    elif bf == "0" and af == "0":
        for f in mfiles:
            print(f)
    elif bf != "0" and af == "0":
        for f in mfiles:
            print(" ".join([bf, f]))
    elif bf != "0" and af != "0":
        for f in mfiles:
            print(" ".join([bf, f, af]))
    
    
    

args = sys.argv
if len(args) == 1:
    print("Usage: looper keyword nkeyword before after")

else:
    _, kw, nkw, bf, af = args
    looper(kw, nkw, bf, af)



In [ ]:
#!/usr/bin/env python
import os
import sys

        
def filter_rows(mfile, col, condition, value):
    
    with open(mfile) as f:
        for l in f:
            h = l.strip().split("\t")
            if condition == ">":
                if float(h[int(col)]) > float(value):
                    print(l.strip())
            elif condition == "<":
                if float(h[int(col)]) < float(value):
                    print(l.strip())
            elif condition == "==":
                if float(h[int(col)]) == float(value):
                    print(l.strip())
            else:
                print("Condition not recognized")
    

args = sys.argv
if len(args) == 1:
    print("Usage: filter_rows mfile col condition value")

else:
    _, mfile, col, condition, value = args
    filter_rows(mfile, col, condition, value)



In [ ]:
#!/usr/bin/env python

# looper

import os
import sys
import argparse

def looper(args):
    kw = args.keyword
    command = args.command
    mode = args.mode
    nkw = args.nkeyword
    adext = args.addextension
    output = args.outfile
   

    if nkw == None:
        mfiles = [x for x in os.listdir("./") if kw in x]
    else:
        mfiles = [x for x in os.listdir("./") if kw in x and nkw not in x]
    
    if mode == "1":
        with open("temploop", "w") as f:            
            for file in mfiles:
                f.write(command + " " + file + "\n")
        os.system("chmod u+x temploop")
        print("Run: ./temploop > %s" % output )
                  
    elif mode == "2":
        for file in mfiles:
            print(command + " " + file + " > " + file + adext)


parser = argparse.ArgumentParser()
parser.add_argument("keyword", type=str, help="Word to be found in file name")
parser.add_argument("command", type=str, help="Command to apply to file")
parser.add_argument("mode", type=str, choices=["1", "2"], help="Type 1: Output of all files redirected to single file. Type 2: Output redirected to multiple files")
parser.add_argument('--nkeyword', default=None, help='Word not to be found in file name')
parser.add_argument('--addextension', default=None, help='Extension to add to files')
parser.add_argument('--outfile', default="Output.txt", help='Output in loop type 1')

args = parser.parse_args()
looper(args)

In [2]:
## Submitter

import os
import sys

_, batch, how_many, coms_files = sys.argv


def submit(num, coms_files):
    
    if os.path.isfile("./coms_submitted"):
        ...
    else:
        ...
    
    
    coms_files = sorted(coms_files)
    
    
    
    
    
    
    
    



ModuleNotFoundError: No module named 'Biopython'

In [ ]:
mfiles = os.listdir("/Users/davin/Downloads/bac_withCPR_trimmed_alignments")



